In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)  # نقوم بتثبيت جذر نموذج الأرقام العشوائية، حتى نحصل على نفس النتائج كل مرة نشغل البرنامج

In [3]:
# تجدون البيانات في رابط أدناه
# https://www.kaggle.com/kazanova/sentiment140/data#
data = pd.read_csv("2477_4140_compressed_training.1600000.processed.noemoticon/training.1600000.processed.noemoticon.csv", encoding="latin-1")

In [4]:
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [5]:
data.columns = ['sentiment', 'tweet id', 'time', 'query', 'username', 'tweet']

In [6]:
data.head()

,sentiment,tweet id,time,query,username,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1599999 non-null  int64 
 1   tweet id   1599999 non-null  int64 
 2   time       1599999 non-null  object
 3   query      1599999 non-null  object
 4   username   1599999 non-null  object
 5   tweet      1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [8]:
data['sentiment'].value_counts()

4    800000
0    799999
Name: sentiment, dtype: int64

In [9]:
# الآن سنقوم بقراءة النصوص و حفظها في قائمتين
# نقوم بإنشاء قائمتين لملئها بالنصوص
negative_texts = []  # السلبية
positive_texts = []  # الإيجابية

In [10]:
import re
def clean_text(text):
    '''
    يجري هنا تجهيز النص من خلال مسح الرموز و محاولة تبسيطه
    :param text: النص المدخل من نوع str
    :return: النص بعد تجهيزه
    '''
    
    text = text.lower()
    # حذف حسابات المستخدمين
    text = re.sub("@[a-z0-9_]+", ' ', text)
    # حذف الروابط
    text = re.sub("[^ ]+\.[^ ]+", ' ', text)
    # حذف البريد الالكتروني
    text = re.sub("[^ ]+@[^ ]+\.[^ ]", ' ', text)
    # حذف الأرقام و الرموز
    text = re.sub("[^a-z\' ]", ' ', text)
    # حذف المسافات الزائدة
    text = re.sub(' +', ' ', text)

    return text

In [11]:
for i in range(len(data)):
    if data['sentiment'][i] == 0:
        negative_texts.append(clean_text(data['tweet'][i]))
    else:
        positive_texts.append(clean_text(data['tweet'][i]))

In [12]:
print("عدد النصوص الإيجابية:")
print(len(positive_texts))
print("عدد النصوص السلبية:")
print(len(negative_texts))

عدد النصوص الإيجابية:
800000
عدد النصوص السلبية:
799999


In [13]:
# ننشئ قائمتين إضافيتين للتصنيفات
# سنرمز للتصنيف الٌيجابي بصفر و التصنيف السلبي بواحد
''' هكذا سيكون تصنيف النص الإيجابي رقم س في قائمة النصوص الإٌيجابية موجودًا في المكان رقم س أيضا في قاذمة تصنيفات النصوص 
الإيجابية، و نفس الشيء بالنسبة للنصوص السلبية'''

positive_labels = [1]*len(positive_texts)  # قائمة تصنيفات النصوص الإيجابية
negative_labels = [0]*len(negative_texts)  # قائمة تصنيفات النصوص السلبية

all_texts = positive_texts + negative_texts  # نضع جميع النصوص في قائمة واحدة
all_labels = positive_labels + negative_labels  # نضع التصنيفات في قائمة واحدة بنفس الترتيب

In [14]:
print("عدد النصوص يساوي عدد التصنيفات؟")
print(len(all_labels) == len(all_texts))  # لابد أن يكون لهما نفس العدد حيث يكون لكل نص تصنيف

عدد النصوص يساوي عدد التصنيفات؟
True


In [15]:
# لاحظ أن النصوص الإيجابية جميعها في البداية و السلبية جميعها في النهاية
# لذا سنقوم بتغيير ترتيب النصوص بشكل عشوائي (مع تصنيفاتها)
# هذه الدالة من مكتبة إس كي ليرن ستقوم بتغيير أماكن النصوص عشوائيا مع مراعاة ارتباط كل نص بتصنيفه
# يمكن إدخال أي عدد من القوائم أو المصفوفات بشرط أن يكون لها نفس الطول
from sklearn.utils import shuffle  # نستدعي الدالة
all_texts, all_labels = shuffle(all_texts, all_labels)  # نشغلها، يجب مراعاة الترتيب للحصول على نتائج صحيحة

In [16]:
# سنكون بحاجة إلى لمعرفة مدى جودة أداء النموذج بعد تدريبه، أي سنحتاج لمجموعة مصنفة من النصوص لنختبر النموذج
# لذا نقسم البيانات لقسمين، قسم للاختبار و قسم للتدريب
# يمثل قسم التدريب ٢٠٪ من حجم النصوص الآصلي
# سنسخدم دالة train_test_split من إس كي ليرن، تقوم هذه الدالة بتقسيم النصوص و التصنيفات
# يمكن إدخال أي عدد من القوائم أو المصفوفات إلى هذه الدالة
# لاحظ أنك لو أدخلت س من القوائم/المصفوفات فإنك ستحصل على ٢*س من القوائم/المصفوفات
from sklearn.model_selection import train_test_split  # نستدعي الدالة
x_train, x_test, y_train, y_test = train_test_split(all_texts, all_labels, test_size=0.20)  # نشغلها، لاحظ الترتيب


In [17]:
# النص الآن جائز للتدريب، ما عدا ￿أن شكله الطبيعي غير مقبول بالنسبة للنموذج، لذا سنقوم باستخراج الميزات
# كما ذكرنا سابقًا، أحد أبرز الطرق لاستخراج الميزات هي تحوسل الكلمة لرقم count vectorization
# سنسخدم نموذج CountVecotrizer من مكتبة sklearn لتحويل الكلمات ￿لأرقام، و هكذا سنكون قد استخرجنا الميزات
# يوفر هذا النموج أيضا بعض طرق اختيار الميزات، كحذف الكلمات النادرة مثلا
# لكننا سنستخدم جميع الميزات بسبب قلة بيانات التدريب
from sklearn.feature_extraction.text import CountVectorizer  # نستدعي count vectorizer
vectorizer = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', min_df=1)  # نقوم بتعريفه، نرغب هنا بأن يستخدم الكلمات كميزات
# نقوم بتدريبه على نصوص التدريب، هذه الخطوة ستجعله يستخرج الكلمات و يرقمها كميزات
vectorizer.fit(x_train)
# ثم نحول نصوص التدريب باستخدامه، هذه الخطوة ستجعله ينشئ مصفوفة أعمدتها الميزات و صفوفها النصوص
# سيملأ كل صف بعدد المرات التي ذكرت فيها الميزة/الكلمة  في اعمود المترتبط في النص المقصود
x_train = vectorizer.transform(x_train)

In [18]:
# لدينا الآن بيانات جاهزة للتدريب، سنحتاج خوارزمية لنستخدمها في تدريب النموذج
# يمكننا الاستعانة بهذه الخريطة الإرشادية لاختيار الخوارزمية الملائمة
# https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
# ذكرت الخريطة الارشادية سابقا أن خوارزميتي LinearSVC و Naive Bayes يوصى بهما للاستخدام مع النصوص
# سنجرب استخدام خوارزمية naive bayes
from sklearn.naive_bayes import MultinomialNB  # نستدعي نموذج خوارزمية naive bayes
model = MultinomialNB()  # نعرف النمذوج باستخدام نموذج خوارزمية إس في إم
model.fit(x_train, y_train)  # نقوم بتدريب النموذج، هذه الخطوة قد تتطلب بعض الوقت عند التشغيل بحسب حجم البيانات

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
# نحتاج ￿لمعرفة مدى جودة التدريب، لذا سنقوم باختبار النموذج على نصوص التدريب
# سنستخدم دالة تقييم لتحسب لنا نسبة صحة إجابات النموذج مقارنة بالتصنيفات الحقيقة
from sklearn.metrics import accuracy_score  # نستدعي دالة التقييم
# نحول نصوص الاختبار إلي أرقام باستخدام count vectorizer الذي تم بناؤه على نصوص التدريب سابقا
x_test = vectorizer.transform(x_test)
# ثم نسخدم النموذج ليتنبأ بتصنيفات النصوص، لاحظ أننا نحتفظ بالتصنيفات الحقيقة أعلاه في المتغير y_test
predictions = model.predict(x_test)
print(" نسبة الصحة باستخدام خوارزمية نايف بيز :")
print(accuracy_score(y_test, predictions))  # نقيم أداء النموذج بإرسال التصنيفات الحقيقية و تصنيفات النموذج
# حصلنا على نتيجة ٧٧٪، أي أن النموذج استطاع أن يتنبأ بشكل صحيح على ٧٧٪ من بيانات الاختبار
# تعتبر هذه النتيجة مقبولة نوعًا ما، لكن يمكننا تجربة خوارزمية أخرى

 نسبة الصحة باستخدام خوارزمية نايف بيز :
0.779878125


In [20]:
# سنجرب إس في إم
# ستسغرق هذه الخوارزمية وقتًا أطول في التدريب
from sklearn.svm import LinearSVC  # نستدعيها
model = LinearSVC()  # نعيد تعريف النموذج باستخدام هذه الخوارزمية
model.fit(x_train, y_train)  # نقوم بتدريب النموذج

C:\Users\nana5\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [21]:
# الآن نقوم باختبار النموذج مرة أخرى
predictions = model.predict(x_test)
print("نسبة الصحة باستخدام خوارزمية إس في إم :")
print(accuracy_score(y_test, predictions))
# حصلنا على نتيجة ٧٨٪ و هي أفضل من نتيجة الخوارزمية السابقة

نسبة الصحة باستخدام خوارزمية إس في إم :
0.786875


In [22]:
# قد ترغب بحفظ النموذج لاستخدامه لاحقًا
# تستخدم مكتبة بيكل pickle لحفظ أي متغير
# يشترط في هذه المكتبة أن يتم حفظ المتغير و تحميله باستخدام نفس النسخة من المكتبات
# سنقوم بحفظ النموذج باستخدامها
# لاحظ أن النموذج لن يقبل أي شكل من المصفوفات غير الذي تدرب عليه، لذا يتوجب استخدام نفس ال vectorizer
# لذا سنقوم بحفظ نفس الvectorizer الذي تم بناوه على بيانات التدريب
import pickle
# نقوم بفتح ملف بالاسم المطلوب للنموذج و نخصص الرمز wb للكتابة
with open('model.pickle', 'wb') as file:
    pickle.dump(model, file)  # تم الحفظ
# نكرر الأمر للvectorizer باسم ملف مختلف طبعًا
# نقوم بفتح ملف بالاسم المطلوب للvectorizer و نخصص الرمز wb للكتابة
with open('vectorizer.pickle', 'wb') as file:
    pickle.dump(vectorizer, file)  # تم الحفظ

In [23]:
# لتحميل النموذج و الvectorizer بعد الحفظ نستخدم نفس المكتبة
# نقوم بفتح ملف النموذج بوضع القراءة rb
with open('model.pickle', 'rb') as file:
    model = pickle.load(file)
# نكرر الأمر لملف ال vectorizer
with open('vectorizer.pickle', 'rb') as file:
    vectorizer = pickle.load(file)

In [24]:
# سنقوم الآن باستخدام النموذج للحصول على تصنيف نص خارجي
# مثلا هذا النص
example_test = 'I am so happy, the picnic was amazing and the weather was great'
# يتوجب تنظيف النص بنفس الطريقة التي تم باستخدامها تنظيف بيانات التدريب
cleaned_example_test = clean_text(example_test)
#ثم سنقوم بتحويل النص إلى أرقام باستخدام الvectorizer الذي أنشئناه سابقًا
# لاحظ أننا سنضع النص داخل مصفوفة ليقبلها، و هذا يعني أنه يمكن إرسال مجموعة من النصوص في مصفوفة كما حدث في بيانات الاختبار سابقا
example_test_vector = vectorizer.transform([cleaned_example_test])
# أخيرا ندخل المصفوفة الناتجة إلى النموذج
example_result = model.predict(example_test_vector)
print("تصنيف الجملة:", example_test)
print(example_result[0])  # سيكون العنصر الأول بطبيعة الحال كوننا لم ندخل سوى جملة واحدة

تصنيف الجملة: I am so happy, the picnic was amazing and the weather was great
1
